<a href="https://colab.research.google.com/github/audwls4545/metaverse/blob/main/bag_of_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 27.9 MB/s eta 0:00:00


In [2]:
from konlpy.tag import Okt

In [3]:
okt = Okt()
text = '나는 오늘 파이썬을 공부하고 있습니다. 파이썬은 정말 재미있는 공부입니다.'

In [7]:
document = text.replace('.', '')
tokenized_document = okt.morphs(document)
tokenized_document

['나',
 '는',
 '오늘',
 '파이썬',
 '을',
 '공부',
 '하고',
 '있습니다',
 '파이썬',
 '은',
 '정말',
 '재미있는',
 '공부',
 '입니다']

In [8]:
word_to_index = {}
bow = []

for word in tokenized_document:
    if word not in word_to_index.keys(): #처음 등장하는 단어
        word_to_index[word] = len(word_to_index)
        bow.insert(len(word_to_index) -1, 1)

    else: #재등장하는 단어
        index = word_to_index.get(word)
        bow[index] += 1

print(word_to_index)
print(bow)

{'나': 0, '는': 1, '오늘': 2, '파이썬': 3, '을': 4, '공부': 5, '하고': 6, '있습니다': 7, '은': 8, '정말': 9, '재미있는': 10, '입니다': 11}
[1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1]


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

text = [
    ['누구나', '한범쯤', '사랑', '웃고'],
    ['누구나', '한번쯤', '사랑', '울고'],
    ['그것', '바로', '사랑', '사랑', '사랑']
]

In [11]:
count_vec = CountVectorizer(tokenizer = lambda x: x, lowercase = False)
result = count_vec.fit_transform(text)
print(count_vec.vocabulary_)

{'누구나': 1, '한범쯤': 7, '사랑': 3, '웃고': 5, '한번쯤': 6, '울고': 4, '그것': 0, '바로': 2}


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [12]:
result.toarray()

array([[0, 1, 0, 1, 0, 1, 0, 1],
       [0, 1, 0, 1, 1, 0, 1, 0],
       [1, 0, 1, 3, 0, 0, 0, 0]])

In [13]:
#tf-idf

In [14]:
import pandas as pd
from math import log
okt = Okt()

data = '''
나는 영어를 좋아합니다.
영어, 수학을 매일 공부합니다.
과학공부를 좋아하고 수학도 좋아 합니다.
영어, 수학, 과학을 좋아 하고 영어는 매일 공부 합니다.
'''

In [17]:
words = okt.morphs(data)
result = []
for temp in words:
    if len(temp) > 1: #한글자 이상인 단어만 저장
        result.append(temp)

result = list(set(result))
result

['과학', '매일', '좋아', '영어', '좋아합니다', '합니다', '좋아하고', '하고', '공부', '수학']

In [18]:
documents = data.split('.')
documents

['\n나는 영어를 좋아합니다',
 '\n영어, 수학을 매일 공부합니다',
 '\n과학공부를 좋아하고 수학도 좋아 합니다',
 '\n영어, 수학, 과학을 좋아 하고 영어는 매일 공부 합니다',
 '\n']

In [19]:
d_list = []
for temp in documents:
    if temp.strip() != '':
        d_list.append(temp.strip())

d_list

['나는 영어를 좋아합니다',
 '영어, 수학을 매일 공부합니다',
 '과학공부를 좋아하고 수학도 좋아 합니다',
 '영어, 수학, 과학을 좋아 하고 영어는 매일 공부 합니다']

In [21]:
N = len(d_list)
def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in d_list:
        df += t in doc
    return log(N/(df+1))

def tfidf(t, f):
    return tf(t, d) * idf(t)

In [22]:
word_list = result
t_result = []

for temp in range(N):
    t_result.append([]) #[[]]
    d = d_list[temp]

    for temp1 in range(len(word_list)):
        t = word_list[temp1]
        t_result[-1].append(tf(t,d))

print(t_result)

[[0, 0, 1, 1, 1, 1, 0, 0, 0, 0], [0, 1, 0, 1, 0, 1, 0, 0, 1, 1], [1, 0, 2, 0, 0, 1, 1, 1, 1, 1], [1, 1, 1, 2, 0, 1, 0, 1, 1, 1]]


In [23]:
df = pd.DataFrame(t_result, columns = word_list)
df

,과학,매일,좋아,영어,좋아합니다,합니다,좋아하고,하고,공부,수학
0,0,0,1,1,1,1,0,0,0,0
1,0,1,0,1,0,1,0,0,1,1
2,1,0,2,0,0,1,1,1,1,1
3,1,1,1,2,0,1,0,1,1,1


In [24]:
tfidf_result = []
for temp in range(len(word_list)):
    t = word_list[temp]
    tfidf_result.append(idf(t))

print(tfidf_result)

[0.28768207245178085, 0.28768207245178085, 0.0, 0.0, 0.6931471805599453, -0.2231435513142097, 0.6931471805599453, 0.28768207245178085, 0.0, 0.0]


In [25]:
df_idf = pd.DataFrame(tfidf_result, index=word_list, columns=['IDF'])
df_idf

,IDF
과학,0.287682
매일,0.287682
좋아,0.000000
영어,0.000000
좋아합니다,0.693147
합니다,-0.223144
좋아하고,0.693147
하고,0.287682
공부,0.000000
수학,0.000000


In [26]:
result = []

for temp in range(N):
    result.append([])
    d = d_list[temp]
    for temp in range(len(word_list)):
        t = word_list[temp]
        result[-1].append(tfidf(t, d))

tfidf_df = pd.DataFrame(result, columns = word_list)
tfidf_df

,과학,매일,좋아,영어,좋아합니다,합니다,좋아하고,하고,공부,수학
0,0.000000,0.000000,0.0,0.0,0.693147,-0.223144,0.000000,0.000000,0.0,0.0
1,0.000000,0.287682,0.0,0.0,0.000000,-0.223144,0.000000,0.000000,0.0,0.0
2,0.287682,0.000000,0.0,0.0,0.000000,-0.223144,0.693147,0.287682,0.0,0.0
3,0.287682,0.287682,0.0,0.0,0.000000,-0.223144,0.000000,0.287682,0.0,0.0
